In [93]:
using Pkg

Pkg.add(["CSV","DataFrames","DecisionTree", "Plots", "JLD2"])
Pkg.add("CategoricalArrays")

   Resolving package versions...


In [1]:
using CSV, DataFrames, Plots, DecisionTree

In [2]:
input_dataset = "https://azuremlexamples.blob.core.windows.net/datasets/iris.csv"
df = CSV.read(download(input_dataset), DataFrame)

,sepal_length,sepal_width,petal_length,petal_width,species
,Float64,Float64,Float64,Float64,String15
1,5.1,3.5,1.4,0.2,Iris-setosa
2,4.9,3.0,1.4,0.2,Iris-setosa
3,4.7,3.2,1.3,0.2,Iris-setosa
4,4.6,3.1,1.5,0.2,Iris-setosa
5,5.0,3.6,1.4,0.2,Iris-setosa
6,5.4,3.9,1.7,0.4,Iris-setosa
7,4.6,3.4,1.4,0.3,Iris-setosa
8,5.0,3.4,1.5,0.2,Iris-setosa
9,4.4,2.9,1.4,0.2,Iris-setosa


In [3]:
using DecisionTree
using CategoricalArrays

In [14]:
features = Matrix(df[:, 1:4])
labels = categorical(df[:, 5])

150-element CategoricalArray{String15,1,UInt32}:
 "Iris-setosa"
 "Iris-setosa"
 "Iris-setosa"
 "Iris-setosa"
 "Iris-setosa"
 "Iris-setosa"
 "Iris-setosa"
 "Iris-setosa"
 "Iris-setosa"
 "Iris-setosa"
 "Iris-setosa"
 "Iris-setosa"
 "Iris-setosa"
 ⋮
 "Iris-virginica"
 "Iris-virginica"
 "Iris-virginica"
 "Iris-virginica"
 "Iris-virginica"
 "Iris-virginica"
 "Iris-virginica"
 "Iris-virginica"
 "Iris-virginica"
 "Iris-virginica"
 "Iris-virginica"
 "Iris-virginica"

In [15]:
# train adaptive-boosted stumps, using 7 iterations
model, coeffs = build_adaboost_stumps(labels, features, 7);

In [16]:
# Quick test
sample = [5.9,3.0,5.1,1.9]
apply_adaboost_stumps(model, coeffs, sample)

CategoricalValue{String15, UInt32} "Iris-virginica"

In [18]:
apply_adaboost_stumps_proba(model, coeffs, sample, categorical(levels(labels)))

3-element Vector{Float64}:
 0.0
 0.2806601104805074
 0.7193398895194926

In [19]:
# run n-fold cross validation for boosted stumps, using 7 iterations and 3 folds
accuracy = nfoldCV_stumps(labels, features, 7, 3)

3×3 Matrix{Int64}:
 6  0  0
 0  5  0
 0  1  9

3×3 Matrix{Int64}:
 7  0  0
 0  9  1
 0  0  4

3×3 Matrix{Int64}:
 9  0  0
 0  5  0
 0  2  5

3×3 Matrix{Int64}:
 6  0  0
 0  6  1
 0  1  7

3×3 Matrix{Int64}:
 7  0  0
 0  6  0
 0  1  7


Accuracy: 0.8095238095238095
Kappa:    0.7083333333333333

Mean Accuracy: 0.9387755102040816

Fold 1
Classes:  String15["Iris-setosa", "Iris-versicolor", "Iris-virginica"]
Matrix:   
Accuracy: 0.9523809523809523
Kappa:    0.9263157894736841

Fold 2
Classes:  String15["Iris-setosa", "Iris-versicolor", "Iris-virginica"]
Matrix:   
Accuracy: 0.9523809523809523
Kappa:    0.925531914893617

Fold 3
Classes:  String15["Iris-setosa", "Iris-versicolor", "Iris-virginica"]
Matrix:   
Accuracy: 0.9047619047619048
Kappa:    0.8551724137931035

Fold 4
Classes:  String15["Iris-setosa", "Iris-versicolor", "Iris-virginica"]
Matrix:   
Accuracy: 0.9047619047619048
Kappa:    0.8561643835616438

Fold 5
Classes:  String15["Iris-setosa", "Iris-versicolor", "Iris-virginica"]
Matrix:   
Accuracy: 0.9523809523809523
Kappa:    0.9285714285714285

Fold 6
Classes:  String15["Iris-setosa", "Iris-versicolor", "Iris-virginica"]
Matrix:   
Accuracy: 0.8571428571428571
Kappa:    0.7789473684210524

Fold 7
Classes:  S

7-element Vector{Float64}:
 0.9523809523809523
 0.9523809523809523
 0.9047619047619048
 0.9047619047619048
 0.9523809523809523
 0.8571428571428571
 0.9047619047619048

In [20]:
using JLD2

In [21]:
mkpath("outputs")
@save "outputs/iris.jld2" model

## Trying to load the model back

In [24]:
using JLD2

trained_model = load("outputs/iris.jld2")

Dict{String, Any} with 1 entry:
  "model" => Ensemble of Decision Trees…

In [25]:
trained_model["model"]

Ensemble of Decision Trees
Trees:      7
Avg Leaves: 2.0
Avg Depth:  1.0